## Atividades Desenvolvidas no Notebook - 03_ingestao_camada_silver

|Sequência|Ação|Detalhamento
|---|---|---|
|SEQ-01 |Função acentuacao|Usada para remover a acentuação de campos do tipo STRING.|
|SEQ-02 / SEQ-03 / SEQ-04 / SEQ-05 / SEQ-06 / SEQ-07 / SEQ-08 / SEQ-09|Ingestão dos Dados da Camada Bronze para Silver|Carga realizada baseada nos dados existentes nas tabelas da camada Bronze.|
|SEQ-02 / SEQ-03 / SEQ-04 / SEQ-05 / SEQ-06 / SEQ-07 / SEQ-08 / SEQ-09|Tratamento de dados antes de inserir/atualizar|Remover acentuação e caracteres especiais, valores nulos onde o datatype foi STRING, informar 'N/A', datatype NUMÉRICO, informar 0 (zero) e datatype DATE/TIMESTAMP, informar '1900-01-01'. Todo os dados do tipo STRING devem estar dados em caixa alta.|
|SEQ-02 / SEQ-03 / SEQ-04 / SEQ-05 / SEQ-06 / SEQ-07 / SEQ-08 / SEQ-09|Identificar a data de alteração ou cadastro|Regra: Caso o registro não exista (caindo na condição INSERT do MERGE), a data de cadastro e data de alteração devem ser a mesma, caso o registro exista, a data de alteração deverá ser alterada.|

In [0]:
%run /estudos/libs/bibliotecas

In [0]:
%sql
MERGE INTO silver.tb_categoria_produto a
USING (
SELECT 
CODIGO_CATEGORIA
,acentuacao(DECRICAO_CATEGORIA) DECRICAO_CATEGORIA
FROM bronze.tb_categoria_produto
) b

ON a.CODIGO_CATEGORIA= b.CODIGO_CATEGORIA

WHEN MATCHED THEN
UPDATE SET 
a.CODIGO_CATEGORIA = b.CODIGO_CATEGORIA
,a.DECRICAO_CATEGORIA = b.DECRICAO_CATEGORIA
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT 
(
CODIGO_CATEGORIA
,DECRICAO_CATEGORIA
,DATA_CADASTRO
,DATA_ALTERACAO
) 
VALUES 
(
b.CODIGO_CATEGORIA
,b.DECRICAO_CATEGORIA
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)

In [0]:
%sql
MERGE INTO silver.tb_cliente a
USING (
SELECT 
CODIGO_CLIENTE
,acentuacao(NOME_CLIENTE) NOME_CLIENTE
FROM bronze.tb_cliente
) b

ON a.CODIGO_CLIENTE= b.CODIGO_CLIENTE

WHEN MATCHED THEN
UPDATE SET 
a.CODIGO_CLIENTE = b.CODIGO_CLIENTE
,a.NOME_CLIENTE = b.NOME_CLIENTE
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT 
(
CODIGO_CLIENTE
,NOME_CLIENTE
,DATA_CADASTRO
,DATA_ALTERACAO
)
VALUES 
(
b.CODIGO_CLIENTE
,b.NOME_CLIENTE
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)


In [0]:
%sql
MERGE INTO silver.tb_forma_pagamento a
USING (
SELECT 
CODIGO_FORMA_PAGAMENTO
,acentuacao(DESCRICAO_FORMA_PAGAMENTO) DESCRICAO_FORMA_PAGAMENTO
FROM bronze.tb_forma_pagamento
) b

ON a.CODIGO_FORMA_PAGAMENTO= b.CODIGO_FORMA_PAGAMENTO

WHEN MATCHED THEN
UPDATE SET 
a.CODIGO_FORMA_PAGAMENTO = b.CODIGO_FORMA_PAGAMENTO
,a.DESCRICAO_FORMA_PAGAMENTO = b.DESCRICAO_FORMA_PAGAMENTO
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT 
(CODIGO_FORMA_PAGAMENTO
,DESCRICAO_FORMA_PAGAMENTO
,DATA_CADASTRO
,DATA_ALTERACAO) 
VALUES 
(b.CODIGO_FORMA_PAGAMENTO
,b.DESCRICAO_FORMA_PAGAMENTO
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)


In [0]:
%sql
MERGE INTO silver.tb_item_venda a
USING (
SELECT 
CODIGO_VENDA
,CODIGO_PRODUTO
,VALOR_UNITARIO
,QUANTIDADE
,VALOR_FINAL
FROM bronze.tb_item_venda
) b

ON a.CODIGO_VENDA= b.CODIGO_VENDA
AND a.CODIGO_PRODUTO= b.CODIGO_PRODUTO

WHEN MATCHED THEN
UPDATE SET 
a.CODIGO_VENDA = b.CODIGO_VENDA
,a.CODIGO_PRODUTO = b.CODIGO_PRODUTO
,a.VALOR_UNITARIO = b.VALOR_UNITARIO
,a.QUANTIDADE = b.QUANTIDADE
,a.VALOR_FINAL = b.VALOR_FINAL
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT 
(
CODIGO_VENDA
,CODIGO_PRODUTO
,VALOR_UNITARIO
,QUANTIDADE
,VALOR_FINAL
,DATA_CADASTRO
,DATA_ALTERACAO
)
VALUES 
(
b.CODIGO_VENDA
,b.CODIGO_PRODUTO
,b.VALOR_UNITARIO
,b.QUANTIDADE
,b.VALOR_FINAL
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)


In [0]:
%sql
MERGE INTO silver.tb_loja a
USING (
SELECT 
CODIGO_LOJA
,acentuacao(NOME_LOJA) NOME_LOJA
,acentuacao(LOCALIDADE_LOJA) LOCALIDADE_LOJA
,TIPO_LOJA
FROM bronze.tb_loja
) b

ON a.CODIGO_LOJA= b.CODIGO_LOJA

WHEN MATCHED THEN
UPDATE SET 
a.CODIGO_LOJA = b.CODIGO_LOJA
,a.NOME_LOJA = b.NOME_LOJA
,a.LOCALIDADE_LOJA = b.LOCALIDADE_LOJA
,a.TIPO_LOJA = b.TIPO_LOJA
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT (
CODIGO_LOJA
,NOME_LOJA
,LOCALIDADE_LOJA
,TIPO_LOJA
,DATA_CADASTRO
,DATA_ALTERACAO
) 
VALUES 
(b.CODIGO_LOJA
,b.NOME_LOJA
,b.LOCALIDADE_LOJA
,b.TIPO_LOJA
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)


In [0]:
%sql

MERGE INTO silver.tb_produto a
USING (SELECT 
PROD.CODIGO_PRODUTO
,acentuacao(PROD.DESCRICAO_PRODUTO) DESCRICAO_PRODUTO
,PROD.CODIGO_CATEGORIA
FROM bronze.tb_produto PROD) b

ON a.CODIGO_PRODUTO= b.CODIGO_PRODUTO

WHEN MATCHED 
THEN
UPDATE SET 
a.CODIGO_PRODUTO = b.CODIGO_PRODUTO
,a.DESCRICAO_PRODUTO = b.DESCRICAO_PRODUTO
,a.CODIGO_CATEGORIA = b.CODIGO_CATEGORIA
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT 
(
CODIGO_PRODUTO
,DESCRICAO_PRODUTO
,CODIGO_CATEGORIA
,DATA_CADASTRO
,DATA_ALTERACAO
) 
VALUES 
(
b.CODIGO_PRODUTO
,b.DESCRICAO_PRODUTO
,b.CODIGO_CATEGORIA
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)


In [0]:
%sql

MERGE INTO silver.tb_venda a
USING (
SELECT 
CODIGO_VENDA
,DATA_VENDA
,CODIGO_CLIENTE
,CODIGO_VENDEDOR
,CODIGO_LOJA
,VALOR_FINAL
,FORMA_PAGAMENTO
,TIPO_PAGAMENTO
FROM bronze.tb_venda
) b

ON a.CODIGO_VENDA= b.CODIGO_VENDA

WHEN MATCHED THEN
UPDATE SET 
a.CODIGO_VENDA = b.CODIGO_VENDA
,a.DATA_VENDA = b.DATA_VENDA
,a.CODIGO_CLIENTE = b.CODIGO_CLIENTE
,a.CODIGO_VENDEDOR = b.CODIGO_VENDEDOR
,a.CODIGO_LOJA = b.CODIGO_LOJA
,a.VALOR_FINAL = b.VALOR_FINAL
,a.FORMA_PAGAMENTO = b.FORMA_PAGAMENTO
,a.TIPO_PAGAMENTO = b.TIPO_PAGAMENTO
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT 
(
CODIGO_VENDA
,DATA_VENDA
,CODIGO_CLIENTE
,CODIGO_VENDEDOR
,CODIGO_LOJA
,VALOR_FINAL
,FORMA_PAGAMENTO
,TIPO_PAGAMENTO
,DATA_CADASTRO
,DATA_ALTERACAO
)
VALUES (
b.CODIGO_VENDA
,b.DATA_VENDA
,b.CODIGO_CLIENTE
,b.CODIGO_VENDEDOR
,b.CODIGO_LOJA
,b.VALOR_FINAL
,b.FORMA_PAGAMENTO
,b.TIPO_PAGAMENTO
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)



In [0]:
%sql

MERGE INTO silver.tb_vendedor a
USING (
SELECT 
CODIGO_VENDEDOR
,acentuacao(NOME_VENDEDOR) NOME_VENDEDOR
FROM bronze.tb_vendedor
) b

ON a.CODIGO_VENDEDOR= b.CODIGO_VENDEDOR

WHEN MATCHED THEN
UPDATE SET 
a.CODIGO_VENDEDOR = b.CODIGO_VENDEDOR
,a.NOME_VENDEDOR = b.NOME_VENDEDOR
,a.DATA_ALTERACAO = from_utc_timestamp(now(),'GMT-3')

WHEN NOT MATCHED
THEN 
INSERT 
(
CODIGO_VENDEDOR
,NOME_VENDEDOR
,DATA_CADASTRO
,DATA_ALTERACAO
) 
VALUES 
(
b.CODIGO_VENDEDOR
,b.NOME_VENDEDOR
,from_utc_timestamp(now(),'GMT-3')
,from_utc_timestamp(now(),'GMT-3')
)

In [0]:
%sql
SELECT *
FROM silver.tb_vendedor